In [66]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import time
import os

from tqdm import tqdm

In [67]:
download_folder = '/Users/alberic/Desktop/Pro/RouskinLab/projects/deep_learning/datasets/PDB/bpSEQ_files_pdbee' # Needs to be absolute !!

chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode (without GUI)
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_experimental_option("prefs", {
    "download.default_directory": download_folder,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})
# Specify the path to the ChromeDriver executable
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver = webdriver.Chrome(options=chrome_options)


In [68]:
import pandas as pd

# This was filtered by:
# - Entry polymer composition = RNA
# - Number of assemblies = 1
# - Number of Distinct Molecular Entities = 1
# - Total Number of Polymer Instances (Chains) = 1

# Louche:
# - 1RXA
PDB_IDs = pd.read_csv('rcsb_pdb_ids.txt').columns.tolist()

In [69]:
for id in tqdm(PDB_IDs):

    try:

        # Open the website
        driver.get('http://rnapdbee.cs.put.poznan.pl/') 

        # Fill the ID name
        entry_box = driver.find_element(By.ID, 'pdbId')
        entry_box.send_keys(id)

        # Click the get button
        driver.find_element(By.XPATH, "//*[@id='tabs-1']/div[3]/div[2]/input[2]").click()
        time.sleep(10)
        # Click the RUN button
        driver.find_element(By.ID, 'commitPdb').click() 
        time.sleep(2)
        # driver.find_element(By.NAME, 'isCt').click()
        driver.find_element(By.NAME, 'isBpseq').click()
        time.sleep(2)
        driver.find_element(By.CLASS_NAME, 'downloadAll').click()
        time.sleep(2)

    except:
        print('Error with ' + id)
    

  4%|▍         | 19/447 [06:05<2:01:52, 17.08s/it]

Error with 1C2W


 24%|██▍       | 108/447 [34:46<1:37:20, 17.23s/it]

Error with 1P5M


 66%|██████▌   | 295/447 [1:35:32<43:54, 17.33s/it]  

...

 99%|█████████▉| 442/447 [2:29:10<01:18, 15.75s/it]

Error with 8SCF


 99%|█████████▉| 443/447 [2:29:22<00:59, 14.79s/it]

Error with 8SCH


100%|██████████| 447/447 [2:30:47<00:00, 20.24s/it]


In [83]:
# Go over all zip flese and unzip them and delete the zip files, then move the file inside the unzipped folder to the main folder
import os
import zipfile
import shutil

for file in os.listdir(download_folder):
    if file.endswith('.zip'):
        with zipfile.ZipFile(os.path.join(download_folder, file), 'r') as zip_ref:
            zip_ref.extractall(download_folder)
        os.remove(os.path.join(download_folder, file))

        save_path = os.path.join(download_folder, file.split('-')[1])
       
        for path, subdirs, files in os.walk(save_path):
            for name in files:
                save_name = os.path.join(download_folder, name)
                while os.path.exists(save_name):
                    save_name = os.path.join(save_name.split('.txt')[0] + '_DUP' + '.txt')
                os.rename(os.path.join(path, name), save_name)
        shutil.rmtree(save_path)



In [84]:
# convert .txt files to .ct files
# And append name of file to the first line

for file in os.listdir(download_folder):
    if file.endswith('.txt'):
        with open(os.path.join(download_folder, file), 'r') as f:
            lines = f.readlines()
            lines[0] = lines[0].split('\n')[0] + ' ' + file.replace('-ct', '').split('.txt')[0] + '\n'
        with open(os.path.join(download_folder, file.replace('-ct', '').split('.txt')[0] + '.ct'), 'w') as f:
            f.writelines(lines)
        os.remove(os.path.join(download_folder, file))

In [92]:
# convert .txt files to .ct files
# And append name of file to the first line

for file in os.listdir(download_folder):
    if file.endswith('.bpseq'):
        with open(os.path.join(download_folder, file), 'r') as f:
            lines = f.readlines()
            # lines[0] = lines[0].split(' ')[0] + ' ' + file.replace('-ct', '').split('.txt')[0] + '\n'
            lines[0] = ' '.join(lines[0].split(' ')[:3])+'\n'
        with open(os.path.join(download_folder, file), 'w') as f:
            f.writelines(lines)


In [90]:
' '.join('1 g 38 1A9L-2D-bpseq'.split(' ')[:3])+'\n'

'1 g 38'

In [85]:
# Convert .txt files to .bpseq files by changinf file name

for file in os.listdir(download_folder):
    if file.endswith('.txt'):
        os.rename(os.path.join(download_folder, file), os.path.join(download_folder, file.replace('-bpseq', '').split('.txt')[0] + '.bpseq'))